In [ ]:

import pandas as pd
import os
import torch.optim as optim
from torchvision import transforms
import numpy as np
import time
from torch.utils.data import Dataset
import h5py
import torchvision.models as models
import torch
import torch.nn as nn
import torch.nn.functional as F
import warnings
from torch.autograd import Variable
from model.model import IClassifier,fusionmodel
warnings.filterwarnings('ignore')
from datasets.dataset import My_features_Dataset
import torch.nn.functional as F               

def validate_one_epoch(test_loader,dataset,net7, net8,net9,device,output_logist = False):###grade_features_index,IDH_features_index,p1_19qs_features_index  
    
    
    net7.eval()
    net8.eval()
    net9.eval()
    results = []
    B = torch.tensor([2])
    # with torch.no_grad():
   

    test_acc_grade_type3 = 0
    test_acc_IDH_type3= 0
    test_acc_p1_19q_type3 = 0
    test_num_grade = 0
    test_num_IDH = 0
    test_num_p1_19q = 0
    for center1, loader in test_loader.items():
        for test_data in loader:
            Grade_Features_type0, IDH_Features_type0, p1_19q_Features_type0_, Grade_Features_type1, IDH_Features_type1, p1_19q_Features_type1_,Grade_Features_type2, IDH_Features_type2, p1_19q_Features_type2_, id, IDH, p1_19q_, grade = test_data

            grade = torch.squeeze(grade)
            grade_features_type0 = torch.squeeze(Grade_Features_type0)
            grade_features_type1 = torch.squeeze(Grade_Features_type1)
            grade_features_type2 = torch.squeeze(Grade_Features_type2)
            
            # grade_features_type2 = torch.cat([grade_features_type0, grade_features_type1], axis=1)
            logits_type3 = net7(grade_features_type0.to(device), grade_features_type1.to(device), grade_features_type2.to(device))
            logits_grade_type3 = torch.squeeze(logits_type3)
            predict_y_type3 = torch.ge(torch.sigmoid(logits_type3), 0.5).float()
            test_acc_grade_type3 += torch.eq(predict_y_type3, grade.float().to(device)).sum().item()

            test_num_grade += 1
        


            IDH = torch.squeeze(IDH)
            IDH_features_type0 = torch.squeeze(IDH_Features_type0)
            IDH_features_type1 = torch.squeeze(IDH_Features_type1)
            IDH_features_type2 = torch.squeeze(IDH_Features_type2)
            logits_type3 = net8(IDH_features_type0.to(device), IDH_features_type1.to(device), IDH_features_type2.to(device))
            logits_idh_type3 = torch.squeeze(logits_type3)
            predict_y_type3 = torch.ge(torch.sigmoid(logits_type3), 0.5).float()
            test_acc_IDH_type3 += torch.eq(predict_y_type3, IDH.float().to(device)).sum().item()

            test_num_IDH += 1



            if center1 != "upenn":
                
                p1_19qs_features_type0 = torch.squeeze(p1_19q_Features_type0_)
                p1_19qs_features_type1 = torch.squeeze(p1_19q_Features_type1_)
                p1_19qs_features_type2 = torch.squeeze(p1_19q_Features_type2_)
               
                logits_type3 = net9(p1_19qs_features_type0.to(device), p1_19qs_features_type1.to(device), p1_19qs_features_type2.to(device))
                logits_p119q_type3 = torch.squeeze(logits_type3)

                if p1_19q_ != B:
                    p1_19qs = torch.squeeze(p1_19q_[~np.isin(p1_19q_, B)])
                    logits_type3_ = torch.squeeze(logits_type3[~np.isin(p1_19q_, B)])
                    predict_y_type3 = torch.ge(torch.sigmoid(logits_type3_), 0.5).float()
                    test_acc_p1_19q_type3 += torch.eq(predict_y_type3, p1_19qs.float().to(device)).sum().item()
                    test_num_p1_19q += 1


            results.append([
                torch.squeeze(id).detach().numpy(), 
                torch.sigmoid(logits_grade_type3).detach().numpy(), grade.detach().numpy(), 
                torch.sigmoid(logits_idh_type3).detach().numpy(), IDH.detach().numpy(), 
                torch.sigmoid(logits_p119q_type3).detach().numpy() if p1_19q_ != B  else None, 
                torch.squeeze(p1_19q_).detach().numpy() ,
                center1
            ])
   


    test_accurate_grade_type3 = test_acc_grade_type3  / test_num_grade
    test_accurate_IDH_type3 = test_acc_IDH_type3 / test_num_IDH

    
    test_accurate_1p_19q_type3 = test_acc_p1_19q_type3  / test_num_p1_19q

       
    print(f'........{dataset:<10}: grade: {test_accurate_grade_type3:.3f}, IDH: {test_accurate_IDH_type3:.3f}, 1p_19q: {test_accurate_1p_19q_type3:.3f}  \n')
    # print(f'Flair , test_accuracy_grade: {test_accurate_grade_type1:.3f}, test_accuracy_IDH:{test_accurate_IDH_type1:.3f}, test_accuracy_1p_19q: {test_accurate_1p_19q_type1:.3f}')
    if output_logist:
        columns = ['ID', 'logits_grade_type2', 'grade', 'logits_idh_type2', 'IDH', 'logits_p119q_type2', 'p1_19qs',"center"]
        results_df = pd.DataFrame(results, columns=columns)
        results_df["dataset"] = dataset
        
        return results_df
    else:
        return test_accurate_grade_type3,test_accurate_IDH_type3,test_accurate_1p_19q_type3

batch_size = 1
nw = min([os.cpu_count(), batch_size if batch_size > 1 else 0, 8])  # number of workers
data_root = "/local/CTimages/brain/code_5mm/data/tumor_slices_2mm/zzu2/03_h5data/features/['T2', 'Flair', 'CET1']"
import pandas as pd
# fold = 1
df_all = pd.DataFrame([])
for fold in (0,1,2,3,4):
    internal_test_image_paths = {
        "zzu1": os.path.join(data_root, f"features_fold{fold}_zzu1.h5")}
    prospective_test_image_paths = {
        "zzu3": os.path.join(data_root, f"features_fold{fold}_zzu3.h5")}
    external_test_image_paths ={
        "xy_gs_nm": os.path.join(data_root, f"features_fold{fold}_xy_gs_nm.h5"),
        "hb": os.path.join(data_root, f"features_fold{fold}_hb.h5"),
        "xj": os.path.join(data_root, f"features_fold{fold}_xj.h5"),
        # "tcga": os.path.join(data_root, f"features_fold{fold}_tcga.h5"),
        "upenn": os.path.join(data_root, f"features_fold{fold}_upenn.h5")
    }
    internal_test_loaders = {}
    for center, path in internal_test_image_paths.items():
        dataset = My_features_Dataset(path)
        loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=False, num_workers=nw, pin_memory=True)
        internal_test_loaders[center] = loader
        print(f"{center} dataloader is ready")
    prospective_test_loaders = {}
    for center, path in prospective_test_image_paths.items():
        dataset = My_features_Dataset(path)
        loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=False, num_workers=nw, pin_memory=True)
        prospective_test_loaders[center] = loader
        print(f"{center} dataloader is ready")

    external_test_loaders = {}
    for center, path in external_test_image_paths.items():
        dataset = My_features_Dataset(path)
        loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=False, num_workers=nw, pin_memory=True)
        external_test_loaders[center] = loader
        print(f"{center} dataloader is ready")
    train_image_path = os.path.join(data_root, f"features_fold{fold}_train.h5")
    train_dataset = My_features_Dataset(train_image_path)
    train_num = len(train_dataset)
    train_loader = torch.utils.data.DataLoader(train_dataset,
                                                    batch_size=batch_size, shuffle=True,
                                                    num_workers=nw, pin_memory=True)


        

    train_dataset_selectfeature = My_features_Dataset(train_image_path,feature_forselection=True)

    train_loader_selectfeature = torch.utils.data.DataLoader(train_dataset_selectfeature,
                                                    batch_size=len(train_dataset_selectfeature), shuffle=False,
                                                num_workers=8)
    val_image_paths = {
        "val": os.path.join(data_root, f"features_fold{fold}_val.h5")}
    val_test_loaders = {}
    for center, path in val_image_paths.items():
        dataset = My_features_Dataset(path)
        loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=False, num_workers=nw, pin_memory=True)
        val_test_loaders[center] = loader
        print(f"{center} dataloader is ready")
    device = "cpu"
    # top_ks = (100,200,250)
    fusion = ['concat','bilinear']

    top_k = 512
    fusion_layres = 'concat'

    relu = True
    print(f"######################################top_k: {top_k}, Fold: {fold}, relu:{relu}######################\n")


    net7 = fusionmodel(n_classes= 1,fusion_layres = fusion_layres,dropout=0.25,scale_dim1=8,gate_path=1,scale_dim2=8,gate_omic=1,skip=True,relu = relu,top_k =top_k).to(device)
    params7 = [p for p in net7.parameters() if p.requires_grad]
    optimizer7 = optim.Adam(params7, lr=0.0001)
    weights_root = f"/local/CTimages/brain/code_5mm/data/tumor_slices_2mm/zzu2/03_h5data/features/['T2', 'Flair', 'CET1']/save_concat_reluTrue/fold{fold}/grade/best_weights"
    weights_path1 = os.path.join(weights_root, os.listdir(weights_root).pop())  
    net7.load_state_dict(torch.load(weights_path1, map_location='cpu'))

    net8 = fusionmodel(n_classes= 1,fusion_layres = fusion_layres,dropout=0.25,scale_dim1=8,gate_path=1,scale_dim2=8,gate_omic=1,skip=True,relu = relu,top_k =top_k).to(device)
    params8 = [p for p in net8.parameters() if p.requires_grad]
    optimizer8 = optim.Adam(params8, lr=0.0001)
    weights_root = f"/local/CTimages/brain/code_5mm/data/tumor_slices_2mm/zzu2/03_h5data/features/['T2', 'Flair', 'CET1']/save_concat_reluTrue/fold{fold}/IDH/best_weights"
    weights_path1 = os.path.join(weights_root, os.listdir(weights_root).pop())  
    net8.load_state_dict(torch.load(weights_path1, map_location='cpu'))

    net9 = fusionmodel(n_classes= 1,fusion_layres = fusion_layres,dropout=0.25,scale_dim1=8,gate_path=1,scale_dim2=8,gate_omic=1,skip=True,relu = relu,top_k =top_k).to(device)
    params9 = [p for p in net9.parameters() if p.requires_grad]
    optimizer9 = optim.Adam(params9, lr=0.0001)
    weights_root = f"/local/CTimages/brain/code_5mm/data/tumor_slices_2mm/zzu2/03_h5data/features/['T2', 'Flair', 'CET1']/save_concat_reluTrue/fold{fold}/p119q/best_weights"
    weights_path1 = os.path.join(weights_root, os.listdir(weights_root).pop())  
    net9.load_state_dict(torch.load(weights_path1, map_location='cpu'))




    df_val = validate_one_epoch(test_loader=val_test_loaders,dataset= "Validation",net7=net7,net8=net8,net9=net9,device= device,output_logist = True)#grade_features_index = grade_features_index,IDH_features_index = IDH_features_index,p1_19qs_features_index = p1_19qs_features_index 
    # print("\n··················································································································")
    # for center, loader in test_loaders[:2].items():
    df_intest = validate_one_epoch(test_loader=internal_test_loaders,dataset= "Internal",  net7=net7,net8=net8,net9=net9,device=device,output_logist = True)#,grade_features_index = grade_features_index,IDH_features_index = IDH_features_index,p1_19qs_features_index = p1_19qs_features_index 
    df_extest = validate_one_epoch(test_loader=external_test_loaders,dataset= "External",  net7=net7,net8=net8,net9=net9,device=device,output_logist = True)
    df_protest = validate_one_epoch(test_loader=prospective_test_loaders,dataset= "Prospective",  net7=net7,net8=net8,net9=net9,device=device,output_logist = True)
        # print("\n··················································································································")
    df_fold = pd.concat([df_val,df_intest,df_extest,df_protest])
    df_fold["Fold"] = f"fold{fold}"
    df_all = pd.concat([df_all,df_fold])

df_all.to_csv("/local/CTimages/brain/code_5mm/data/tumor_slices_2mm/zzu2/03_h5data/features/['T2', 'Flair', 'CET1']/results.csv")

zzu1 dataloader is ready
zzu3 dataloader is ready
xy_gs_nm dataloader is ready
hb dataloader is ready
xj dataloader is ready
upenn dataloader is ready
val dataloader is ready
######################################top_k: 512, Fold: 0, relu:True######################

........Validation: grade: 0.842, IDH: 0.863, 1p_19q: 0.766  

........Internal  : grade: 0.837, IDH: 0.867, 1p_19q: 0.800  

........External  : grade: 0.847, IDH: 0.899, 1p_19q: 0.743  

........Prospective: grade: 0.849, IDH: 0.891, 1p_19q: 0.833  

zzu1 dataloader is ready
zzu3 dataloader is ready
xy_gs_nm dataloader is ready
hb dataloader is ready
xj dataloader is ready
upenn dataloader is ready
val dataloader is ready
######################################top_k: 512, Fold: 1, relu:True######################

........Validation: grade: 0.832, IDH: 0.899, 1p_19q: 0.854  

........Internal  : grade: 0.840, IDH: 0.856, 1p_19q: 0.873  

........External  : grade: 0.846, IDH: 0.896, 1p_19q: 0.810  

........Prospective: gra